In [ ]:
from __future__ import annotations # To prevent Subscript for class "list" will generate runtime exception; enclose type annotation in quotes
import seaborn as sns
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import scipy
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from Kmeans import *
import math
%matplotlib inline
# pd.options.display.max_rows = 4000

In [ ]:
df = pd.read_csv('ml-1m/users.dat', delimiter='::', names=['UserID','Gender','Age','Occupation','Zip-code'])
df

In [ ]:
le=LabelEncoder()
df['Gender']=le.fit_transform(df['Gender'])
df

In [ ]:
df['Zip-code'] = df['Zip-code'].apply(lambda x: int(x.split('-')[0]) if type(x)==str else x)
df['Zip-code'] = df['Zip-code'].apply(lambda x: int(str(x)[0]))

In [ ]:
df['Zip-code'] = df['Zip-code'].astype('int64')

In [ ]:
df.describe()

In [ ]:
zipcode=df['Zip-code']
df = pd.get_dummies(
    data=df, # dataframe to one hot encode
    columns=["Zip-code"])
df = pd.get_dummies(
    data=df, # dataframe to one hot encode
    columns=["Occupation"])

In [ ]:
df.drop(['UserID'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
for i in df.columns[2:]:
	lst=[]
	for j in df[i]:
		if(j):
			lst.append(1)
		else:
			lst.append(0)
	df[i]=lst

df

In [ ]:
df['Age'] = df['Age'].apply(lambda x: x/56)

In [ ]:
df

In [ ]:
ratings=pd.read_csv('ml-1m/ratings.dat', delimiter='::', names=['UserID','MovieID','Rating','TimeStamp'])

In [ ]:
ratings.describe()

In [ ]:
mean=ratings.loc[:, 'Rating'].mean()
std=ratings.loc[:, 'Rating'].std()
rating = ratings['Rating'].apply(lambda x: (x-mean)/ std)
min=ratings.loc[:, 'Rating'].min()
max=ratings.loc[:, 'Rating'].max()
rating = ratings['Rating'].apply(lambda x: (x-min)/ (max-min))
ratings['Rating']=rating
# min=ratings.loc[:, 'TimeStamp'].min()
# ratings['TimeStamp'] = ratings['TimeStamp'].apply(lambda x: x-min)

In [ ]:
ratings

In [ ]:
ratings.sort_values(by=['TimeStamp'],inplace=True)

In [ ]:
tmp=list(ratings['TimeStamp'])
plt.plot(range(len(tmp)),tmp)
plt.show()

In [ ]:
ratings.describe()

In [ ]:
movies=pd.read_csv('ml-1m/movies.dat', delimiter='::', names=["Title","Genres"],  encoding="ISO-8859-1")

In [ ]:
movies['Genres'] = movies['Genres'].apply(lambda x: x.split("|"))

In [ ]:
movies

In [ ]:
Genres=["Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
tmp=[[] for _ in range(6040)]
for i in Genres:
    df[i]=tmp
df['LastLog']=np.nan

In [ ]:
df

In [ ]:
# pd.options.display.max_rows = 4000
for j in ratings.index:
    uid,mid,rating,time=ratings.iloc[j]
    uid=int(uid)
    mid=int(mid)
    time=int(time)
    try:
        genres=movies.at[mid-1,"Genres"]
        for i in genres:
            temp = df.iloc[uid-1][i]
            if(len(temp)==0 or time-temp[1]>=31536000):
                temp=[rating,time]
            else:
                temp=[rating+temp[0]*(1-(time-temp[1])/31536000),time]
            df.at[uid-1,i]=temp
            df.at[uid-1,"LastLog"]=time
    except:
        pass

In [ ]:
df

In [ ]:
df_off=df.copy()
for i in Genres:
    for j in df_off.index:
        temp=df_off.at[j,i]
        try:
            if(df_off.at[j,"LastLog"]<=31536000):
                df_off.at[j,i]=temp[0]*(temp[1]/df_off.at[j,"LastLog"])
            else:
                if(df_off.at[j,"LastLog"]-temp[1]>=31536000):
                    df_off.at[j,i]=0
                else:
                    df_off.at[j,i]=temp[0]*(1-(df_off.at[j,"LastLog"]-temp[1])/31536000)
        except:
            df_off.at[j,i]=np.nan

In [ ]:
df_off

In [ ]:
df_off.isna().sum().sum()

In [ ]:
df_off.drop(["LastLog"],axis=1,inplace=True)
df_off['Zip-code']=zipcode
grouped = df_off.groupby(['Age','Gender','Zip-code'])
grouped.count()

In [ ]:
for i in Genres:
    df_off[i] = grouped[i].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df_off.isna().sum().sum()

In [ ]:
df_off.drop("Zip-code",axis=1,inplace=True)

In [ ]:
df_off

In [ ]:
ToNormalize=df_off[Genres]
min=ToNormalize.min(axis=1)
max=ToNormalize.max(axis=1)
# print(min,max)
for i in ToNormalize.index:
    for j in Genres:
        ToNormalize.at[i,j]=(ToNormalize.at[i,j]-min[i])/(max[i]-min[i])
# ToNormalize=ToNormalize.apply(lambda x:(x-min)/max, axis=1)
for i in Genres:
    df_off[i]=ToNormalize[i]

In [ ]:
df_off

In [ ]:
df_off.to_csv('preprocessed.csv')

In [ ]:
from SVD import *
u,s,sigma,V_trans = ReducedSVD(df.to_numpy(),0,0)

In [ ]:
s

In [ ]:
u,s,sigma,V_trans = ReducedSVD(df.to_numpy(),0,4)

In [ ]:
s

In [ ]:
num_cluster = 6

In [ ]:
df = pd.DataFrame(u@sigma@V_trans,columns=df.columns)

In [ ]:
df

In [ ]:
def cluster_to_matrix(X_train,y_train,num_cluster):
    k=Kmeans(K=num_cluster,iter=150,p=2,random_state=random_state)
    y_pred,_=k.predict(np.array(X_train),choice=0)
    k.centroids=np.array(k.centroids)
    empty_cluster = []
    for idx,i in enumerate(k.clusters):
        if len(i)==0:
            empty_cluster.append(idx)
    X_train2 = pd.concat([X_train,y_train,pd.DataFrame(y_pred,columns=["label"],dtype='int')],axis="columns")
    df_cluster_rep = X_train2.groupby(['label'], as_index=False).mean()
    df_cluster_rep.drop(labels="label",axis="columns",inplace=True)
    matrix = df_cluster_rep.to_numpy()
    final_matrix = []
    j = 0
    for i in range(num_cluster):
        if i in empty_cluster:
            final_matrix.append(np.zeros(25))
        else:
            final_matrix.append(matrix[j])
            j+=1

    return np.array(final_matrix),k
    return matrix,k

In [ ]:
X = df[[df.columns[0],df.columns[1],df.columns[2],df.columns[3],df.columns[4]]]
y = df.drop(labels=["Data Structures and Algorithms","Computer Architecture\n","Discrete Mathematics\n","Economics","Programming-2"],axis="columns")

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.05, shuffle=True,random_state=random_state)

In [ ]:
matrix,kmeans_model = cluster_to_matrix(X_train,y_train,num_cluster)

In [ ]:
matrix.shape

In [ ]:
def predict(datapoint,kmeans,matrix_clean):
    '''
        @input: Takes in a datapoint, kmeans object, and information matrix
        @output: Returns: rating sorted with index for column name mapping
    '''
    y_pred = kmeans.predictPoint(datapoint)
    inferred_full = matrix_clean[y_pred]
    inferred = inferred_full[5:]
    sorted_ratings = []
    for idx,element in enumerate(inferred):
        sorted_ratings.append([element,idx+5])
    sorted_ratings.sort()
    sorted_ratings.reverse()
    return sorted_ratings

In [ ]:
X_test.to_numpy()[0][:5]
sorted_ratings = predict(X_test.to_numpy()[0][:5],kmeans_model,matrix)
s="Top 5 PREDICTED recommended electives are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings[i][1]]+"\n"
print(s)

In [ ]:
inferred = y_test.to_numpy()[0]
for idx,element in enumerate(inferred):
    sorted_ratings.append([element,idx+5])
sorted_ratings.sort()
sorted_ratings.reverse()
s="Top 5 ACTUAL recommended electives are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings[i][1]]+"\n"
print(s)

In [ ]:
def predict_for_dataset(X_test,kmeans_instance,matrix_clean):
    ''' Runs out inference and return predicted ratings of subjects
        @input: X_test,kmeans object, clean_matrix after SVD
    '''
    X_test = X_test.to_numpy()
    y_pred = []
    for i in range(X_test.shape[0]):
        datapoint =X_test[i][:5]
        y_pred.append(predict(datapoint,kmeans_instance,matrix_clean))
    return np.array(y_pred)

In [ ]:
y_pred = predict_for_dataset(X_test,kmeans_model,matrix)
y_pred.shape

In [ ]:
def accuracy(y_pred,y_test):
    ''' Finds out % common subject in top 5 rated subjects (predicted and actual)
        @input: predicted values and test dataset values
        @output: accuracy value
    '''
    y_test = y_test.to_numpy()
    sorted_ratings_actual = []
    for i in range(y_test.shape[0]):
        inferred = y_test[i]
        t = []
        for idx,element in enumerate(inferred):
            t.append([element,idx+5])
        t.sort()
        t.reverse()
        sorted_ratings_actual.append(np.array(t))
    sorted_ratings_actual = np.array(sorted_ratings_actual)

    total = 0
    correct = 0


    for i in range(len(sorted_ratings_actual)):
        total+=5
        temp = sorted_ratings_actual[i][:5]
        # if i ==1:
        #     print(temp)
        for j in range(5):
            for k in range(len(temp)):
                if(y_pred[i][j][1]==temp[k][1]):correct+=1

    perecent_correct_for_datapoint = []

    for i in range(len(sorted_ratings_actual)):
        per_correct = 0
        temp = sorted_ratings_actual[i][:5]
        # if i ==1:
        #     print(temp)
        for j in range(5):
            for k in range(len(temp)):
                if(y_pred[i][j][1]==temp[k][1]):per_correct+=1
        perecent_correct_for_datapoint.append(per_correct/5)
    # print("Total Entries received: ", len(sorted_ratings_actual)*5," Total Correct: ",correct)
    return 100*(correct/total),perecent_correct_for_datapoint,sorted_ratings_actual


In [ ]:
overall_val,per_point_val,y_sorted_actual = accuracy(y_pred,y_test)

In [ ]:
overall_val

In [ ]:
per_point_val

#### Grid Search 

In [ ]:
from SVD import *
from sklearn.model_selection import train_test_split
accuracy_list_gs = []
def grid_search(df):
    for i in range(4,9):
        for j in range(0,i):
            # print(i,j)
            u,s,sigma,V_trans = ReducedSVD(df.to_numpy(),0,j)
            df = pd.DataFrame(u@sigma@V_trans,columns=df.columns)
            X = df[[df.columns[0],df.columns[1],df.columns[2],df.columns[3],df.columns[4]]]
            y = df.drop(labels=["Data Structures and Algorithms","Computer Architecture\n","Discrete Mathematics\n","Economics","Programming-2"],axis="columns")
            X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.05, shuffle=True,random_state=random_state)
            matrix,kmeans_model = cluster_to_matrix(X_train,y_train,i)
            if(matrix.shape[0]<i):
                break
            y_pred = predict_for_dataset(X_train,kmeans_model,matrix)
            overall_val,per_point_val,y_sorted_actual = accuracy(y_pred,y_train)
            accuracy_list_gs.append([overall_val,('Number of cluster',i),('SV removed',j)])
    accuracy_list_gs.sort(reverse=True)
grid_search(df)

In [ ]:
accuracy_list_gs

In [ ]:
k = 6
to_remove = 1